In [0]:
import pandas as pd
from datetime import datetime,timedelta
from google.colab import files

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Cleaning Movies

In [0]:
df=pd.read_excel("/content/Movie_dataset_3subcategories.xlsx",sheet_name="All years combined")
#df_2019=df[df['Year']==2019][['Year','Opening','Date','Title','Unnamed: 4','Director','Cast','Studio (production house)','Subcategories']]
df_2019=df[['Year','Opening','Date','Title','Unnamed: 4','Director','Cast','Studio (production house)','Subcategories 1','Subcategories 2','Subcategories 3']]
df_2019=df_2019.rename(columns={"Unnamed: 4":"uni_code","Studio (production house)":"Studio"})
df_2019=df_2019[df_2019['uni_code']!=""]

In [0]:
df_2019.head()

,Year,Opening,Date,Title,uni_code,Director,Cast,Studio,Subcategories 1,Subcategories 2,Subcategories 3
0,2018,October,5,Andhadhun,2iVYI99VGaw,Sriram Raghavan,"Ayushman Khurana, Tabu, Radhika Apte",Viacom 18 Motion Pictures,4,4,3
1,2019,March,8,Badla,mSlgu8AQAd4,Sujoy Ghosh,"Amitabh Bachchan, Taapsee Pannu","Red Chillies Entertainment, Azure Entertainment",4,4,3
2,2019,January,11,Uri: The Surgical Strike,VVY3do673Zc,Aditya Dhar,"Vicky Kaushal, Mohit Raina, Paresh Rawal, Yami...",RSVP Movies,4,4,3
3,2018,March,23,Hichki,nLSaCFlXn-g,Siddharth P Malhotra,Rani Mukerji,Yash Raj Films,4,4,3
4,2018,February,9,Pad Man,-K9ujx8vO_A,R. Balki,"Akshay Kumar, Sonam Kapoor, Radhika Apte","Columbia Pictures KriArj Entertainment, Hope P...",4,4,3


In [0]:
#Creating the threshold datettime

df_2019['Month']=[datetime.strptime(x,"%B").month for x in df_2019['Opening']]

df_2019['Month']=df_2019['Month'].astype(str)
df_2019['Month']=["0"+ x if len(x)==1 else x for x in df_2019['Month']]
df_2019['Date']=df_2019['Date'].astype(str)
df_2019['Date']=["0"+ x if len(x)==1 else x for x in df_2019['Date']]

df_2019['Datef']=df_2019['Year'].astype(str)+"-"+df_2019['Month'].astype(str)+"-"+df_2019['Date'].astype(str)

df_2019['Datef']=[datetime.strptime(x,"%Y-%m-%d") for x in df_2019['Datef']]
df_2019['Threshold']=[x-timedelta(days=7) for x in df_2019['Datef']]
df_2019['lower_thres']=[x-timedelta(days=21) for x in df_2019['Datef']]
df_2019['Week']=[x.isocalendar()[1] for x in df_2019['Datef']]
del df_2019['Datef']
del df_2019['Month']
del df_2019['Year']

df_2019=df_2019.fillna("")

In [0]:
#parsing cast,director,studio to list
df_2019['Cast']=[x+", " for x in df_2019['Cast']]
df_2019['Cast 1']=[x.split(",")[0] for x in df_2019['Cast']]
df_2019['Cast 1']=[x.lower() for x in df_2019['Cast 1']]
df_2019['Cast 1']=[x.strip() for x in df_2019['Cast 1']]

df_2019['Cast 2']=[x.split(",")[1] for x in df_2019['Cast']]
df_2019['Cast 2']=[x.lower() for x in df_2019['Cast 2']]
df_2019['Cast 2']=[x.strip() for x in df_2019['Cast 2']]

df_2019['Director']=[x.split(",")[0] for x in df_2019['Director']]
df_2019['Studio']=[x.split(",")[0] for x in df_2019['Studio']]

In [0]:
df_2019.to_csv("all_celeaned_model-2.csv")
files.download("all_celeaned_model-2.csv")

## Developing Features

In [0]:
#installing required packages
pip install emot

In [0]:
#importing required libraries
import nltk.data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')

import pandas as pd
import numpy as np
import scipy as sp
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import words
from itertools import combinations
import random as rd
from emot.emo_unicode import UNICODE_EMO, EMOTICONS
#from googletrans import Translator
from datetime import datetime
from nltk.stem import WordNetLemmatizer 
import ast
from scipy import sparse

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [0]:
parent_list=df_2019.copy()
L_zip =list(zip(parent_list['uni_code'],parent_list['Threshold'],parent_list['lower_thres']))

### Utility Functions

In [0]:
#Declaring all necessary functions
setofwords = set(words.words())

#Convert emojis to text
def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, " " + "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()))
        #text = text.replace(emot,"")
    return text

#apply emoji convert & translate hindi to english
def convert_lang(x,date1,date2): 
  path="/content/drive/My Drive/project/allmovies/"+x+".csv"
  try:
    df=pd.read_csv(path,engine='python')
  except:
    df=pd.read_csv(path,engine='c')
  del df['Unnamed: 0']
  df=df.dropna()
  df['pub_dates']=[datetime.strptime(x.split("T")[0], "%Y-%m-%d") for x in df['pub_dates']]
  comments=df[(df['pub_dates']<date1) & (df['pub_dates']>date2)]['comments']
  comments=[convert_emojis(x) for x in comments]
  return comments

#remove punctuations, special characters and stop words from the text
def clean_comments(comment):
  # removing all non aplhabetic characters
  clean_comment=re.sub("[^a-zA-Z]+"," ",comment)

  #lowercase & split
  clean_comment = clean_comment.lower()
  clean_comment = clean_comment.split()

  #remove stop words & join
  from nltk.corpus import stopwords
  clean_comment = [w for w in clean_comment if not w in stopwords.words("english")]
  lemmatizer = WordNetLemmatizer()
  clean_comment=[lemmatizer.lemmatize(w) for w in clean_comment]
  clean_comment=[x for x in clean_comment if x in setofwords]
  clean_comment=" ".join( clean_comment)

  return clean_comment

#get words that have highest tfidf score
def get_topwords(cleaned_comments):
  print("getting_top_words")
  cv=CountVectorizer(max_features=5000)
  word_count_vector=cv.fit_transform(cleaned_comments)
  tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
  tfidf_transformer.fit(word_count_vector)
  df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"])
  df_idf=df_idf.sort_values(by=["idf_weights"],ascending=False).iloc[0:200]
  return list(df_idf.index)

In [0]:
final_vectors = pd.DataFrame(columns=['videoId','top_words'])
L_ex=[]
i=0
cleaned_comm=[]
for x,date1,date2 in L_zip:
  print(i)
  try:
    proc_comments = convert_lang(x,date1,date2)
    print("cleaning now")
    cleaned_comments=[clean_comments(comment) for comment in proc_comments]
    cleaned_comm=cleaned_comm+[".".join(cleaned_comments)]
    i=i+1
  except:
    print("excepted")
    print(i)
    L_ex=L_ex+[i]
    i=i+1
    continue
  

vectorizer = CountVectorizer(analyzer = "word",tokenizer = None, preprocessor = None, stop_words = None,max_features = 200)
train_data_features = vectorizer.fit_transform(cleaned_comm)
X_counts=train_data_features.toarray()
transformer=TfidfTransformer(smooth_idf=False)
tfidf=transformer.fit_transform(X_counts)
X_tfidf=tfidf.toarray()
X_tfidf2=np.where(X_tfidf>0,1,X_tfidf)

0
cleaning now
1
cleaning now
2
cleaning now
3
cleaning now
4
cleaning now
5
cleaning now
6
cleaning now
7
cleaning now
8
cleaning now
9
cleaning now
10
cleaning now
11
cleaning now
12
cleaning now
13
cleaning now
14
cleaning now
15
cleaning now
16
cleaning now
17
cleaning now
18
cleaning now
19
cleaning now
20
cleaning now
21
cleaning now
22
cleaning now
23
cleaning now
24
cleaning now
25
cleaning now
26
cleaning now
27
cleaning now
28
cleaning now
29
cleaning now
30
cleaning now
31
cleaning now
32
cleaning now
33
cleaning now
34
cleaning now
35
cleaning now
36
cleaning now
37
cleaning now
38
cleaning now
39
cleaning now
40
cleaning now
41
cleaning now
42
cleaning now
43
cleaning now
44
cleaning now
45
cleaning now
46
cleaning now
47
cleaning now
48
cleaning now
49
cleaning now
50
cleaning now
51
cleaning now
52
cleaning now
53
cleaning now
54
cleaning now
55
cleaning now
56
cleaning now
57
cleaning now
58
cleaning now
59
cleaning now
60
cleaning now
61
cleaning now
62
cleaning now
63

In [0]:
for_tensor=df_2019.copy()
for_tensor=for_tensor.drop(index=L_ex)
#the following are needed when read from csv directly
#for_tensor['Cast']=[ast.literal_eval(x) for x in for_tensor['Cast']]
#for_tensor['Studio']=[ast.literal_eval(x) for x in for_tensor['Studio']]
#for_tensor['Director']=[ast.literal_eval(x) for x in for_tensor['Director']]
for_tensor.head()

,Opening,Date,Title,uni_code,Director,Cast,Studio,Subcategories 1,Subcategories 2,Subcategories 3,Threshold,lower_thres,Week,Cast 1,Cast 2
0,October,05,Andhadhun,2iVYI99VGaw,Sriram Raghavan,"Ayushman Khurana, Tabu, Radhika Apte,",Viacom 18 Motion Pictures,4,4,3,2018-09-28,2018-09-14,40,ayushman khurana,tabu
1,March,08,Badla,mSlgu8AQAd4,Sujoy Ghosh,"Amitabh Bachchan, Taapsee Pannu,",Red Chillies Entertainment,4,4,3,2019-03-01,2019-02-15,10,amitabh bachchan,taapsee pannu
2,January,11,Uri: The Surgical Strike,VVY3do673Zc,Aditya Dhar,"Vicky Kaushal, Mohit Raina, Paresh Rawal, Yami...",RSVP Movies,4,4,3,2019-01-04,2018-12-21,2,vicky kaushal,mohit raina
3,March,23,Hichki,nLSaCFlXn-g,Siddharth P Malhotra,"Rani Mukerji,",Yash Raj Films,4,4,3,2018-03-16,2018-03-02,12,rani mukerji,
4,February,09,Pad Man,-K9ujx8vO_A,R. Balki,"Akshay Kumar, Sonam Kapoor, Radhika Apte,",Columbia Pictures KriArj Entertainment,4,4,3,2018-02-02,2018-01-19,6,akshay kumar,sonam kapoor


In [0]:
#final_list=list(for_tensor['top_words'])
#words_set=set([item for sublist in final_list for item in sublist])


#final_cast
#for_tensor['Cast']=[x[0:1] for x in for_tensor['Cast']]
final_cast=list(for_tensor['Cast 1'])
final_cast=final_cast+list(for_tensor['Cast 2'])
cast_set=set(final_cast)


#director
final_dir=list(for_tensor['Director'])
#dir_set=set([item for sublist in final_dir for item in sublist])


#production house
#for_tensor['Studio']=[x[0:1] for x in for_tensor['Studio']]
final_pro=list(for_tensor['Studio'])
#pro_set=set([item for sublist in final_pro for item in sublist])
words_set=set([])
words_set.update(cast_set)
words_set.update(final_pro)
words_set.update(final_dir)

words_set=list(words_set)

word_ids=list(range(0,len(words_set)))

dict_cov=dict(zip(words_set,word_ids))

In [0]:
rows=[]
columns=[]
data=[]
for i in range(0,len(for_tensor)):
  print(i)
  #words=for_tensor.iloc[i]['top_words']
  #words=[dict_cov[x] for x in words]
  #columns=columns+words

  cast=[for_tensor.iloc[i]['Cast 1'],for_tensor.iloc[i]['Cast 2']]
  cast=[dict_cov[x] for x in cast]
  columns=columns+cast
  
  dire=[for_tensor.iloc[i]['Director']]
  dire=[dict_cov[x] for x in dire]
  columns=columns+dire

  pro=[for_tensor.iloc[i]['Studio']]
  pro=[dict_cov[x] for x in pro]
  columns=columns+pro

  n=len(dire)+len(cast)+len(pro)
  row=[i]*n
  vals=[1]*n

  rows=rows+row
  data=data+vals

mat=sparse.coo_matrix((data,(rows,columns)),shape=(len(for_tensor),len(words_set)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [0]:
for_mat2=mat.toarray()
s_t=np.concatenate((X_tfidf2,for_mat2),axis=1)

In [0]:
X_tfidf2.shape
org_mat = pd.DataFrame(data=X_tfidf, columns=['c'+str(i) for i in range(X_tfidf.shape[1])])
org_mat.to_csv("For_randomforest.csv")

In [0]:
files.download()

In [0]:
cat_result1 = np.array(for_tensor['Subcategories 1'])
cat_result2 = np.array(for_tensor['Subcategories 2'])
cat_result3 = np.array(for_tensor['Subcategories 3'])
#org_mat = mat.toarray()
org_mat_df = pd.DataFrame(data=s_t, columns=['c'+str(i) for i in range(s_t.shape[1])])
org_mat_df.to_csv("/content/drive/My Drive/project/Results/model2_data_200.csv")
cat_result_df1 = pd.DataFrame({'t': cat_result1})
cat_result_df1.to_csv("/content/drive/My Drive/project/Results/model2_data_target_1.csv")
cat_result_df2 = pd.DataFrame({'t': cat_result2})
cat_result_df2.to_csv("/content/drive/My Drive/project/Results/model2_data_target_2.csv")
cat_result_df3 = pd.DataFrame({'t': cat_result3})
cat_result_df3.to_csv("/content/drive/My Drive/project/Results/model2_data_target_3.csv")